In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from PIL import UnidentifiedImageError
from tensorflow.keras import backend as K
from tensorflow.keras.applications import VGG19

# Create a nested folder for the generated images
output_folder = './Data/fits_filtered9/augmented_images/gan_output2'
os.makedirs(output_folder, exist_ok=True)

# Step 1: Load and Preprocess Dataset
def load_images_from_folder(folder, image_size=(128, 128)):
    images = []
    valid_files = 0
    invalid_files = 0
    for filename in os.listdir(folder):
        try:
            img = load_img(os.path.join(folder, filename), target_size=image_size)
            if img is not None:
                images.append(img_to_array(img))
                valid_files += 1
            else:
                invalid_files += 1
        except (UnidentifiedImageError, OSError):
            print(f"Skipping file {filename}, as it is not a valid image.")
            invalid_files += 1
    print(f"Loaded {valid_files} valid images, skipped {invalid_files} invalid images.")
    return np.array(images)

dataset = load_images_from_folder('./Data/fits_filtered9/augmented_images')
print(f"Dataset shape: {dataset.shape}")
if dataset.size == 0:
    raise ValueError("No valid images found in the dataset. Please check the image files.")
dataset = (dataset - 127.5) / 127.5  # Normalize to [-1, 1]

# Step 2: Build the GAN

# Generator
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(512 * 8 * 8, activation="relu", input_dim=100))
    model.add(layers.Reshape((8, 8, 512)))
    model.add(layers.BatchNormalization(momentum=0.8))

    model.add(layers.UpSampling2D())  # 16x16
    model.add(layers.Conv2D(256, kernel_size=3, padding="same"))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation("relu"))

    model.add(layers.UpSampling2D())  # 32x32
    model.add(layers.Conv2D(128, kernel_size=3, padding="same"))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation("relu"))

    model.add(layers.UpSampling2D())  # 64x64
    model.add(layers.Conv2D(64, kernel_size=3, padding="same"))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation("relu"))

    model.add(layers.UpSampling2D())  # 128x128
    model.add(layers.Conv2D(3, kernel_size=3, padding="same"))
    model.add(layers.Activation("tanh"))
    return model

# Discriminator
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, input_shape=(128, 128, 3), padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, kernel_size=3, strides=2, padding="same"))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(512, kernel_size=3, strides=2, padding="same"))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Wasserstein loss
def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

# Perceptual loss
def perceptual_loss(y_true, y_pred):
    vgg = VGG19(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
    vgg.trainable = False
    model = tf.keras.Model(inputs=vgg.input, outputs=vgg.get_layer('block5_conv4').output)
    y_true_features = model(y_true)
    y_pred_features = model(y_pred)
    return K.mean(K.square(y_true_features - y_pred_features))

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss=wasserstein_loss, optimizer=tf.keras.optimizers.Adam(0.0001, 0.5), metrics=['accuracy'])

# Build the generator
generator = build_generator()

# The generator takes noise as input and generates images
z = layers.Input(shape=(100,))
img = generator(z)

# For the combined model, only the generator is trained
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model (stacked generator and discriminator)
combined = tf.keras.Model(z, valid)
combined.compile(loss=[wasserstein_loss], optimizer=tf.keras.optimizers.Adam(0.0001, 0.5))

# Step 3: Train the GAN

# Training parameters
epochs = 100
batch_size = 64
save_interval = 500

X_train = dataset
half_batch = int(batch_size / 2)

for epoch in range(epochs):
    # Train Discriminator
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = X_train[idx]

    noise = np.random.normal(0, 1, (half_batch, 100))
    gen_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, -np.ones((half_batch, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator
    noise = np.random.normal(0, 1, (batch_size, 100))
    valid_y = np.ones((batch_size, 1))

    g_loss = combined.train_on_batch(noise, valid_y)

    print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}] [G loss: {g_loss}]")

    if epoch % save_interval == 0:
        noise = np.random.normal(0, 1, (25, 100))
        gen_imgs = generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale images 0 - 1

        fig, axs = plt.subplots(5, 5)
        cnt = 0
        for i in range(5):
            for j in range(5):
                axs[i, j].imshow(gen_imgs[cnt])
                axs[i, j].axis('off')
                cnt += 1
        plt.savefig(os.path.join(output_folder, f'epoch_{epoch}.png'))
        plt.close()

# Generate final samples
noise = np.random.normal(0, 1, (10, 100))
gen_imgs = generator.predict(noise)
gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale images 0 - 1

for i in range(10):
    plt.imshow(gen_imgs[i])
    plt.axis('off')
    plt.savefig(os.path.join(output_folder, f'final_{i}.png'))
    plt.close()



Skipping file gan_output, as it is not a valid image.
Skipping file gan_output2, as it is not a valid image.
Skipping file gan_output3, as it is not a valid image.
Loaded 32 valid images, skipped 3 invalid images.
Dataset shape: (32, 128, 128, 3)


1/1 [==============================] - 1s 581ms/step


0 [D loss: 0.12245893478393555, acc.: 34.375] [G loss: 0.5129153728485107]
1/1 [==============================] - 0s 272ms/step
1 [D loss: -0.13370664790272713, acc.: 1.5625] [G loss: 0.5676045417785645]
1/1 [==============================] - 0s 266ms/step
2 [D loss: -0.30531225353479385, acc.: 0.0] [G loss: 0.6513931751251221]
1/1 [==============================] - 0s 272ms/step
3 [D loss: -0.3629514202475548, acc.: 3.125] [G loss: 0.7515891790390015]
1/1 [==============================] - 0s 271ms/step
4 [D loss: -0.40273595601320267, acc.: 7.8125] [G loss: 0.8006399869918823]
1/1 [==============================] - 0s 282ms/step
5 [D loss: -0.45866600796580315, acc.: 0.0] [G loss: 0.8